In [192]:
!pip install pandas numpy

In [191]:
# imports
import pandas as pd
import numpy as np

# **Assignment - Data Cleaning**

Group: #45

Isaac Lafond - 300191954

## **Dataset 1 - Clean Data checking**

In [ ]:
# Read dataset from public github repo
df1 = pd.read_csv("https://raw.githubusercontent.com/IsaacLafond/CSI-4142---Fundamentals-of-Data-Science/main/Assignment%202/datasets/dirty_cafe_sales.csv")

### **1) Data Type errors**

Data type check:

In this test we verify that the data entered in the columns follow a coherent datatype and count the occurrences of incorrect datatypes.

In [ ]:
# Parameters
columns1 = ["Item", "Quantity", "Price Per Unit", "Total Spent", "Payment Method", "Location", "Transaction Date"]
# Notice transaction ID is not include as all the values are present and unique

test_column1 = "Transaction Date"

In [ ]:
# Checker code
df1.info()
# List of invalid value types
invalid = ["ERROR", "UNKNOWN"]

def num_valid_categories(col, invalid):
  # remove values from given column that aren't invalid, not na and not null and returning the count the of column
  return df1[df1[col].isin(invalid) | df1[col].isna() | df1[col].isnull()][col].value_counts() #[3]

def num_valid_numeric(col):
  # Convert column to numeric and force error to be NAN which aren't counted in final column and return count
  return df1[pd.to_numeric(df1[col], errors="coerce").isna()][col].value_counts() #[1]

def num_valid_date(col):
  return df1[pd.to_datetime(df1[col], errors="coerce").isna()][col].value_counts() #[2]

# Dictionnary of predicate the count error values
valid_values = {
    "Item": num_valid_categories("Item", invalid),
    "Quantity": num_valid_numeric("Quantity"),
    "Price Per Unit": num_valid_numeric("Price Per Unit"),
    "Total Spent": num_valid_numeric("Total Spent"),
    "Payment Method": num_valid_categories("Payment Method", invalid),
    "Location": num_valid_categories("Location", invalid),
    "Transaction Date": num_valid_date("Transaction Date")
}

print("=====")
# print(f"\"{test_column1}\" has {valid_values[test_column1]} invalid values")
result = valid_values[test_column1]
print(f"Total invalid values: {result.sum()}")
result

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB
=====
Total invalid values: 301


,count
Transaction Date,
UNKNOWN,159
ERROR,142


### **2) Range errors**

Range check:

In this test, we will verify the range of a numerical value. The range is the minimum and maximum values that an attribute can have.

In [ ]:
columns2 = ["Quantity", "Price Per Unit", "Total Spent"]
# Attribute selection
test_attribute = "Quantity"
# Minimum (e.g. 0$)
minimum = 0
# Maximum (e.g. 200K$)
maximum = 4

In [ ]:
# Checker code
df1_num = df1[[test_attribute]].copy()
df1_num[test_attribute] = pd.to_numeric(df1_num[test_attribute], errors="coerce")

#[4]
print("Values below minimum:\n", df1_num[df1_num[test_attribute] < minimum].value_counts()) #[4]
print("=====")
print("Values above maximum:\n", df1_num[df1_num[test_attribute] > maximum].value_counts()) #[4]

Values below minimum:
 Series([], Name: count, dtype: int64)
=====
Values above maximum:
 Quantity
5.0         2013
Name: count, dtype: int64


### **3) Format errors**

Format check:

In this test we verify the that data follows a coherent and consistent format.

In [ ]:
# Categorical and numerical can follow a format and therefore aren't evaluated here
columns3 = ["Transaction ID", "Transaction Date"]

# Transaction ID format: TXN_*******
# Transaction Date format: YYYY-MM-DD
format_patterns = {
    "Transaction ID": "^TXN_[0-9]{7}$",
    "Transaction Date": "^[0-9]{4}-[0-9]{2}-[0-9]{2}$"
}

test_column3 = "Transaction Date"

In [ ]:
# Checker code

# Return the value counts of rows in the column that don't follow the desired format
df1[~df1[test_column3].str.match(format_patterns[test_column3], na=False)][test_column3].value_counts() #[5]

,count
Transaction Date,
UNKNOWN,159
ERROR,142


### **4) Consistency errors**

Consistency check:

In this test we verify that data is logically consistent between columns.

In [ ]:
# Checker code (only 1 possible parameter, see below)

# Quantity, Price Per Unit and Total Spent are the only column logically linked
# Therefore we report the consistency of these columns by returning a data frame of inconsistent rows
df1_temp = df1.copy()

# Convert columns to numeric (to handle possible string values)
df1_temp["Quantity"] = pd.to_numeric(df1_temp["Quantity"], errors="coerce")
df1_temp["Price Per Unit"] = pd.to_numeric(df1_temp["Price Per Unit"], errors="coerce")
df1_temp["Total Spent"] = pd.to_numeric(df1_temp["Total Spent"], errors="coerce")

mask = df1_temp["Quantity"] * df1_temp["Price Per Unit"] != df1_temp["Total Spent"]

df1_temp.loc[mask, ["Quantity", "Price Per Unit", "Total Spent"]]

# [6]

,Quantity,Price Per Unit,Total Spent
2,4.0,1.0,NaN
20,NaN,4.0,20.0
25,3.0,4.0,NaN
31,2.0,1.0,NaN
42,2.0,1.5,NaN
...,...,...,...
9984,NaN,4.0,4.0
9988,5.0,3.0,NaN
9993,2.0,4.0,NaN
9996,3.0,NaN,3.0


### **5) Uniqueness errors**

Uniqueness check:

In this test we verify unique identitfier aren't reused more than once in the dataset.

In [ ]:
# Checker code (only 1 possible parameter, see below)

# Typically this type of check can be executed on identfier such as ID, emails or usernames
# In this case the only identifier in the dataset is the transaction ID
tranID_column = df1["Transaction ID"]

# Get a list of all the unique value from the Transaction ID column and print its size
print(f"Number of unique values: {tranID_column.unique().size}")
# See the value counts of all the transaction ID (all 1)
tranID_column.value_counts()

Number of unique values: 10000


,count
Transaction ID,
TXN_1961373,1
TXN_4831525,1
TXN_1228927,1
TXN_6486912,1
TXN_3447069,1
...,...
TXN_5680238,1
TXN_7766134,1
TXN_2617257,1


### **6) Presence errors**

Presence check:

In this check we verify the presence of invalid (error) or unfilled values in columns.

In [ ]:
# Paramters
columns6 = ["Item", "Quantity", "Price Per Unit", "Total Spent", "Payment Method", "Location", "Transaction Date"]

# Test column
test_column6 = "Item"

In [ ]:
# Checker code

# Function to return value counts of invalid values
def invalid_values(col):
  return df1[df1[col].isin(["ERROR", "UNKNOWN"]) | df1[col].isna() | df1[col].isnull()][col].value_counts(dropna=False) #[6]

# Count the number of "ERROR", "UNKNOWN", NaN or Null value
for col in columns6:
  print(invalid_values(col))
  print("=====")

Item
UNKNOWN    344
NaN        333
ERROR      292
Name: count, dtype: int64
=====
Quantity
UNKNOWN    171
ERROR      170
NaN        138
Name: count, dtype: int64
=====
Price Per Unit
ERROR      190
NaN        179
UNKNOWN    164
Name: count, dtype: int64
=====
Total Spent
NaN        173
UNKNOWN    165
ERROR      164
Name: count, dtype: int64
=====
Payment Method
NaN        2579
ERROR       306
UNKNOWN     293
Name: count, dtype: int64
=====
Location
NaN        3265
ERROR       358
UNKNOWN     338
Name: count, dtype: int64
=====
Transaction Date
NaN        159
UNKNOWN    159
ERROR      142
Name: count, dtype: int64
=====


### **7) Length errors**

Length check:

In this check we verify that each value in the columns are of the appropriate character length.

In [ ]:
# Parameters
# List all the columns (numerical and categorical data columns not in scope)
columns7 = ["Transaction ID", "Transaction Date"]

# Appropriate length for each applicable column
# ID: TXN_*******, Date: YYYY-MM-DD
length_patterns = {
    "Transaction ID": 11,
    "Transaction Date": 10
}

# Test column
test_column7 = "Transaction Date"

In [ ]:
# Checker code

# Check the length of each value in the test column and show the values with incorect lengths
df1[df1[test_column7].str.len() != length_patterns[test_column7]][test_column7].value_counts(dropna=False)

,count
Transaction Date,
NaN,159
UNKNOWN,159
ERROR,142


### **8) Look-up errors**

Look-up check:

In this test we verify that the values found in the dataset only match the accepted possible values dictated by the dataset.

In [ ]:
# Parameters
"""
Dataset description:

   Cafe menu:
Item      Price($)
Coffee	  2
Tea	      1.5
Sandwich	4
Salad	    5
Cake	    3
Cookie	  1
Smoothie	4
Juice	    3

Accepted payment methods:
- Cash
- Credit Card
- Digital Wallet

Possible location:
- In-store
- Takeaway
"""

# Create dict of the acceptable values per column
accepted_values = {
    "Item": ["Coffee", "Tea", "Sandwich", "Salad", "Cake", "Cookie", "Smoothie", "Juice"],
    "Price Per Unit": ["2.0", "1.5", "4.0", "5.0", "3.0", "1.0", "4.0", "3.0"],
    "Payment Method": ["Cash", "Credit Card", "Digital Wallet"],
    "Location": ["In-store", "Takeaway"]
}

# Test attribute
test_column8 = "Item"

In [ ]:
# Checker code

# Show count values of values that aren't in the accepted values list for the column in question
df1[~df1[test_column8].isin(accepted_values[test_column8])][test_column8].value_counts(dropna=False)

,count
Item,
UNKNOWN,344
NaN,333
ERROR,292


### **9) Exact duplicate errors**

Exact duplicate check:

In this test we verify the presence of exact duplicates or rows whose values are completely identical.

In [ ]:
# Parameters
"""
Since in the previous exercises we've identified that each ID is unique,
the dataset won't have any true exact duplicates (see below). However,
if we remove the ID from consideration and search for exact duplicates
using a subset of columns excluding the ID, duplicates may be found.
"""
# Subset excluding the Transaction ID
columns9 = ["Item", "Quantity", "Price Per Unit", "Total Spent", "Payment Method", "Location", "Transaction Date"]

In [ ]:
# Checker code

# Check for true exact duplicates and print the count
print(f"Number of true exact duplicates: {df1[df1.duplicated()].count()}")
print("\n=====\n")

# Check for exact duplicates in the subset of column not including the Transaction ID as they're all unique
print("Exact duplicates ignoring the Transactions ID:")
df1[df1.duplicated(subset=columns9, keep=False)] #[8][9]

Number of true exact duplicates: Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

=====

Exact duplicates ignoring the Transactions ID:


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
27,TXN_5695074,Juice,4,3.0,12.0,Credit Card,Takeaway,2023-04-10
115,TXN_1001832,Salad,2,5.0,10.0,Cash,Takeaway,UNKNOWN
126,TXN_9646452,Juice,5,3.0,15.0,Credit Card,NaN,2023-09-16
300,TXN_1623966,Tea,2,1.5,3.0,Cash,Takeaway,UNKNOWN
...,...,...,...,...,...,...,...,...
9898,TXN_8999360,Juice,3,3.0,9.0,Cash,Takeaway,2023-09-27
9909,TXN_2112522,Sandwich,5,4.0,20.0,Digital Wallet,Takeaway,2023-04-30
9930,TXN_4428252,Sandwich,3,4.0,12.0,NaN,Takeaway,2023-10-31
9967,TXN_8563793,Juice,4,3.0,12.0,NaN,In-store,2023-03-26


### **10) Near duplicate errors**

Near duplicate errors:

In this test we verify the presence of near duplicate or rows whose values are nearly identical (Only vary in a few non critical columns values).

In [ ]:
# Parameters
import ipywidgets as widgets
from IPython.display import display

# Create a multi-selection dropdown to allow the selection of the subset of columns
dropdown = widgets.SelectMultiple(
    options=df1.columns,
    value=[],
    description="Columns",
    disabled=False
)

display(dropdown)

# Checker code

# Selected values
def on_selection_change(change):
  # Get selected columns
  selected_subset = list(change["new"])
  # Check for duplicates in the subset of column
  display(df1[df1.duplicated(subset=selected_subset, keep=False)]) #[8][9]

dropdown.observe(on_selection_change, names="value")

# [10]

SelectMultiple(description='Columns', options=('Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total …

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9994,TXN_7851634,UNKNOWN,4,4.0,16.0,NaN,NaN,2023-01-08
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
27,TXN_5695074,Juice,4,3.0,12.0,Credit Card,Takeaway,2023-04-10
115,TXN_1001832,Salad,2,5.0,10.0,Cash,Takeaway,UNKNOWN
126,TXN_9646452,Juice,5,3.0,15.0,Credit Card,NaN,2023-09-16
300,TXN_1623966,Tea,2,1.5,3.0,Cash,Takeaway,UNKNOWN
...,...,...,...,...,...,...,...,...
9898,TXN_8999360,Juice,3,3.0,9.0,Cash,Takeaway,2023-09-27
9909,TXN_2112522,Sandwich,5,4.0,20.0,Digital Wallet,Takeaway,2023-04-30
9930,TXN_4428252,Sandwich,3,4.0,12.0,NaN,Takeaway,2023-10-31
9967,TXN_8563793,Juice,4,3.0,12.0,NaN,In-store,2023-03-26


## **Dataset 2 - Imputation**

In [187]:
# Read dataset from public github repo
df2 = pd.read_csv("https://raw.githubusercontent.com/IsaacLafond/CSI-4142---Fundamentals-of-Data-Science/main/Assignment%202/datasets/workout_fitness_tracker_data.csv")

### **Attribute 1 - Mood after workout**

For this attribute we simulate "missing completely at random" (MCAR) as if participants speratically report their mood after workouts or simply forgot.

In [219]:
# Simulate missing data

# Set seed for reproducibility
np.random.seed(45)

# Get 2 copy for this exercise and original values
original_values1 = df2['Mood After Workout'].copy()
df2_1 = df2.copy()

# Randomly remove 25% of the values in the "Mood After Workout" column by setting them to Null
num_to_null = int(len(df2_1) * 0.25)  # Calculate 25%
random_indices = np.random.choice(df2_1.index, size=num_to_null, replace=False)  # Random indices

# Set selected rows to NaN
df2_1.loc[random_indices, "Mood After Workout"] = np.nan
# [11]

In [229]:
# Imputation implementation
# Implement univariate mode imputation
mood_mode = df2_1["Mood After Workout"].mode()[0]
df2_1.loc[random_indices, "Mood After Workout"] = mood_mode

# Get imputed values
imputed_values = df2_1.loc[random_indices, "Mood After Workout"]

In [231]:
# Imputation evaluation

# Compute categorical imputation accuracy
correct_imputations = (imputed_values == original_values1[random_indices]).sum()
accuracy = correct_imputations / num_to_null

print("Mode used for imputation:", mood_mode)
print(f"Categorical Imputation Accuracy: {accuracy:.2%}")

Mode used for imputation: Energized
Categorical Imputation Accuracy: 13.08%


# **References**

[1] https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html

[2] https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

[3] ChatGPT: Prompt = "Pandas allows for removing values from a dataset based on condition like the following: df[df["age"] > 0] to remove age values 0 or less. Can you write a similar condition which removes values that are null or equal to the values in the following list: ["ERROR", "UNKNOWN"]?"

[4] Week 4 Data Quality/Cleaning Part 1

[5] https://pandas.pydata.org/docs/reference/api/pandas.Series.str.match.html

[6] ChatGPT: Prompt = "I need to create a consistency error check for a dataset of coffee house transactions. The dataset contains a column for the quantity of the item purchased, the price per unit of the item and the total amount spent. Can you help me write a check that return a dataframe that only contains rows of these 3 columns where quantity*price per unit doesn't equal total spent?"

[7] https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html

[8] Week 4 Data Quality/Cleaning Part 2

[9] https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html

[10] ChatGPT: Prompt =
1. "Given a list of columns say: ["Item", "Quantity", "Price Per Unit", "Total Spent", "Payment Method", "Location", "Transaction Date"]. What's the most efficient way to allow the user to select a subset of the option found in the list?"
2. "ipywidgets seems like it would work! How can I use the selected subset in the duplicated pandas method?"

[11] ChatGPT: Prompt = "How can I randomly remove 25% of the values in a column of a pandas dataset by setting them to null in order to practice imputation methods?"